In [1]:
%matplotlib notebook

import numpy as np
import astropy.units as u
from astropy import time
from astropy.time import Time
from astropy.time import TimeDelta
from astropy.coordinates import solar_system_ephemeris
from poliastro import iod
from poliastro.bodies import Sun
from poliastro.twobody import Orbit
from poliastro.ephem import get_body_ephem
from poliastro.constants import GM_mars, GM_earth, R_mars, R_earth
from timeit import default_timer as timer


solar_system_ephemeris.set("jpl")


def velocity(number):
    start = timer()
    DeltaDL = 0
    y = []
    z = []
    q = 0
    f = 0    
    flight_1 = []
    flight_2 = []
    dvbudget = (number/1000) * u.km / u.s 
    Initial_date_launch_1 = time.Time('2017-08-1 12:00', scale='utc') # Initial Launch Date
    
    # TO MARS    

    
    for DeltaDL in range (0, 360, 3): # Delta Launch Date
        
        DeltaL = TimeDelta(DeltaDL, format = 'jd') 
        date_launch = Initial_date_launch_1 + DeltaL # sets time of launch date       
        
        
        for tof in range(60, 270, 2): # range of tof loop
        
            TOF = TimeDelta(tof, format='jd')
            date_arrival = date_launch + TOF # sets time of arrival
        
            # planet velocity and position vecors for Launch Date and Date Arrival
            r0, vv_e = get_body_ephem("earth", date_launch) 
            vv_e  = vv_e.to(u.km / u.second)
            rf, vv_m = get_body_ephem("mars", date_arrival) 
            vv_m  = vv_m.to(u.km / u.second)
            
            # solving LAMBERT'S PROBLEM
            (va, vb), = iod.lambert(Sun.k, r0, rf, TOF) 
            ss0_trans = Orbit.from_vectors(Sun, r0, va, date_launch) # creating orbit         
            incl = ss0_trans.inc # inclination of orbit
            ssref_trans = Orbit.from_vectors(Sun, r0, vv_e, date_launch)
            inclref = ssref_trans.inc
            incl_actual = np.absolute(incl-inclref)
            
            # accounting for Earth Orbit
            v_1 = va - vv_e # delta v vector
            dv1 = np.sqrt(v_1.dot(v_1)) # magnitude of delta v                    
            radius_E_i = (R_earth.value + 200*1000) # radius of parking orbit
            v_parking_E  = np.sqrt(GM_earth.value/radius_E_i)*(u.m/u.s) #velocity in parking orbit
            # Hyperbolic excess speed
            v_hyperbolic_E = np.sqrt((dv1.value*1000)**2+(2*GM_earth.value)/(radius_E_i))*(u.m/u.s) 
            Real_Delta_v_E = v_hyperbolic_E - v_parking_E 
            # actual delta v
            
            
            # accounting for Mars Orbit
            v_2 = vv_m - vb
            dv2 = np.sqrt(v_2.dot(v_2))       
            Radius_Parking_M = (R_mars.value + 150*1000) # Radius
            v_parking_M  = np.sqrt(GM_mars.value/ Radius_Parking_M )*u.m/u.s
            v_hyperbolic_M = np.sqrt((dv2.value*1000)**2+(2*GM_mars.value)/( Radius_Parking_M ))*u.m/u.s
            dv_2 = v_hyperbolic_M - v_parking_M 
            M_Hyperbolic_incl = np.sin(incl_actual/4)**2 # Accounting for Inclination
            Real_Delta_v_M =  np.sqrt(dv_2**2+(4*v_parking_M*v_hyperbolic_M*M_Hyperbolic_incl))
            # Delta V Velocity accounting for inclination                          
            tdv = (  Real_Delta_v_E + Real_Delta_v_M )
            # total velocity needed for trajectory to Mars
            
            q += 1
            print(q,end='\r') # Progress
            
            # Condition for storing orbit
            if (dvbudget - (5750 *  u.m / u.s) )  >= tdv: # 5750 m/s minimum velocity to get back
                y = np.array([date_launch, TOF, tdv])
                flight_1 = np.append(flight_1,[y[:]])         
    
    velocity.flight_1 = flight_1
    
    end = timer()
    print(end - start) # Time for calculations
    
    
    
    
    # to Earth  
    Initial_date_launch_2 = time.Time('2018-05-01 12:00', scale='utc')

    for DeltaDL_2 in range(0, 186, 3):
        WT = TimeDelta(DeltaDL_2, format = 'jd')
        date_launch_2 =  Initial_date_launch_2 + WT
        
        for tof2 in range(60, 270, 2):

            TOF_2 = TimeDelta(tof2, format='jd')
            date_arrival_2 = date_launch_2 + TOF_2                       


            r0_2, vv_m_2 = get_body_ephem("mars", date_launch_2) 
            vv_m_2  = vv_m_2.to(u.km / u.second)
            rf_2, vv_e_2 = get_body_ephem("earth", date_arrival_2 ) 
            vv_e_2  = vv_e_2.to(u.km / u.second)
            # planet velocity and position vecors

            (va_2, vb_2), = iod.lambert(Sun.k, r0_2, rf_2, TOF_2) # solving LAMBERT'S PROBLEM
            ss0_trans_2 = Orbit.from_vectors(Sun, r0_2, va_2, date_launch_2) # creating orbit  
            ssref_trans_2 = Orbit.from_vectors(Sun, rf_2, vv_e_2, date_launch_2)
            incl_2 = ss0_trans_2.inc # inclination
            inclref_2 = ssref_trans_2.inc
            incl_2 = ss0_trans_2.inc
            incl_actual2 = np.absolute(incl_2-inclref_2)


            # accounting for Mars Orbit
            v_3 = vv_m_2 - va_2
            dv3 = np.sqrt(v_3.dot(v_3)) # magnitude                    
            radius_M_i = (R_mars.value + 150*1000)
            v_parking_M_2  = np.sqrt(GM_mars.value/radius_M_i)*(u.m/u.s) #velocity in parking orbit
            v_hyperbolic_M_2 = np.sqrt((dv3.value*1000)**2+(2*GM_mars.value)/(radius_M_i))*(u.m/u.s) 
            dv_3 = v_hyperbolic_M_2 - v_parking_M_2            
            M_Hyperbolic_incl_2 = np.sin(incl_actual2/4)**2 
            Real_Delta_v_M_2 =  np.sqrt(dv_3**2+(4*v_parking_M_2*v_hyperbolic_M_2*M_Hyperbolic_incl_2))
            # = delta v accounting for change in incl


            # accounting for Earth Orbit
            v_4 = vb_2 - vv_e_2 
            dv4 = np.sqrt(v_4.dot(v_4)) #magnitudes        
            Radius_Parking_E = (R_earth.value + 200*1000)
            v_parking_E_2  = np.sqrt(GM_earth.value/ Radius_Parking_E )*u.m/u.s
            v_hyperbolic_E_2 = np.sqrt((dv4.value*1000)**2+(2*GM_earth.value)/( Radius_Parking_E ))*u.m/u.s
            Real_Delta_v_E_2 = v_hyperbolic_E_2 - v_parking_E_2 #delta v to get in circular orbit
            
            tdv_2 = Real_Delta_v_E_2 + Real_Delta_v_M_2
            f += 1
            print(f,end='\r') # Progress
            
            if (dvbudget - (6150 *  u.m / u.s) )  >= tdv_2:
                z = np.array([date_launch_2, TOF_2, tdv_2])
                flight_2 = np.append(flight_2,[z[:]])
                
    end = timer()
    print(end - start) # Time for calculations
    
    
    velocity.flight_2 = flight_2

In [2]:
velocity(21000)

511.5746626412043
789.0912680679992


In [13]:
def linkvectors(vect_1, vect_2):
    
    vect_1 = vect_1.reshape( int((vect_1.size)/3) , 3)
    vect_1 =  vect_1[np.argsort(vect_1[:,0])]    
    vect_2 = vect_2.reshape( int((vect_2.size)/3) , 3)
    vect_2 =  vect_2[np.argsort(vect_2[:,0])]
    print(vect_1[0,0])
    print(vect_2[0,0])
    v1 = []
    v2 = []
    q = 0   
    a = False
    b = False
    k = 0
    reuse = []
    date_launch_2 = 0
    TOF_2 = 0
    tdv_2 = 0
    date_launch_1 = 0
    TOF_1 = 0
    tdv_1 = 0   
    start = timer()
    
    k = (vect_2.size*vect_1.size)/9
    print(k)
    
    for x in range (0, int((vect_2.size)/3), 1):
        v2 = np.append(v2,[vect_2[x,:]])           

    if len(v2) == False:
        print("no results")
        return
        
    v2 = v2.reshape( int((v2.size)/3) , 3)            
    print(v2.size)
    for y in range(0, int((v2.size)/3), 1):
        
        q += 1  
        print(q/(v2.size/3)*100, end = '\r')
        if y == 0 or v2[y,0] != v2[y-1,0] :
            reuse = []
            
            for z in range(0, int((vect_1.size)/3), 1):
                a = False
                b = False                          
                pause = timer()     
                
                if (( v2[y,0] - TimeDelta(20, format = 'jd') ) >= (vect_1[z,0] + vect_1[z,1])) == True:
                    a = True

                if a & (( v2[y,0] - TimeDelta(100, format = 'jd') ) <= (vect_1[z,0] + vect_1[z,1])) == True:
                    b = True
                    reuse = np.append(reuse,vect_1[z,:])
            if len(reuse) != 0:
                reuse = reuse.reshape( int((reuse.size)/3) , 3)         
        
        if len(reuse) != 0:
            for c in range(0, int((reuse.size)/3), 1):   
            
                if (( v2[y,2] + reuse[c,2]) <= 21 * u.km / u.s) == True:
                    deltav = v2[y,2] + reuse[c,2]
                    WT = v2[y,0] - (reuse[c,0] + reuse[c,1])
                    # Wait Time = date_launch_2 - (date_launch_1 + tof_1)

                    TotTime = reuse[c,1] + WT + v2[y,1]
                    # tof_1 + WT + tof_2
                    date_launch_2 = v2[y,0]
                    TOF_2 = v2[y,1]
                    tdv_2 = v2[y,2]
                    date_launch_1 = reuse[c,0]
                    TOF_1 = reuse[c,1]
                    tdv_1 = reuse[c,2]                    
                    v1 = np.append(v1,[date_launch_1, TOF_1, tdv_1, date_launch_2, TOF_2, tdv_2, WT, TotTime, deltav])
          
    
    print("..............................................")
    
    
    if len(v1) != 0:
        v1 = v1.reshape( int((v1.size)/9) , 9)
        v1 =  v1[np.argsort(v1[:,7])]        
        print("finished")
        end = timer()
        print(end-start)
    else:
        print("no results")
        return
    
    linkvectors.v1 = v1

In [ ]:
linkvectors(velocity.flight_1, velocity.flight_2)

In [1]:
q = 0
linkvectors.v1 =  linkvectors.v1[np.argsort(linkvectors.v1[:,7])]
linkvectors.v1 =  linkvectors.v1[np.argsort(linkvectors.v1[:,6])]  
             
with open("Orbits_old" + str(0) +".csv", "w") as out_file:
    for i in range (int(linkvectors.v1.size/9)):        
        print(q/(linkvectors.v1.size/3)*100, end = '\r')
        q += 1
        out_string = ""
        out_string = str(i) + "\n"
        out_string += str(linkvectors.v1[i,0]) + "\n"
        out_string += str(linkvectors.v1[i,1]) + "\n"
        out_string += str(linkvectors.v1[i,2]) + "\n"
        out_string += str(linkvectors.v1[i,3]) + "\n"
        out_string += str(linkvectors.v1[i,4]) + "\n"
        out_string += str(linkvectors.v1[i,5]) + "\n"
        out_string += str(linkvectors.v1[i,6]) + "\n"
        out_string += str(linkvectors.v1[i,7]) + "\n"
        out_string += str(linkvectors.v1[i,8]) + "\n"
        out_string += "\n"
        out_file.write(out_string)

linkvectors.v1 =  linkvectors.v1[np.argsort(linkvectors.v1[:,8])]          
linkvectors.v1 =  linkvectors.v1[np.argsort(linkvectors.v1[:,7])]  
             
with open("Orbits_old" + str(1) +".csv", "w") as out_file:
    for i in range (int(linkvectors.v1.size/9)):
        print(q/(linkvectors.v1.size/3)*100, end = '\r')
        q += 1
        out_string = ""
        out_string = str(i) + "\n"
        out_string += str(linkvectors.v1[i,0]) + "\n"
        out_string += str(linkvectors.v1[i,1]) + "\n"
        out_string += str(linkvectors.v1[i,2]) + "\n"
        out_string += str(linkvectors.v1[i,3]) + "\n"
        out_string += str(linkvectors.v1[i,4]) + "\n"
        out_string += str(linkvectors.v1[i,5]) + "\n"
        out_string += str(linkvectors.v1[i,6]) + "\n"
        out_string += str(linkvectors.v1[i,7]) + "\n"
        out_string += str(linkvectors.v1[i,8]) + "\n"
        out_string += "\n"
        out_file.write(out_string)

    
linkvectors.v1 =  linkvectors.v1[np.argsort(linkvectors.v1[:,8])]  
             
with open("Orbits_old" + str(2) +".csv", "w") as out_file:
    for i in range (int(linkvectors.v1.size/9)):
        print(q/(linkvectors.v1.size/3)*100, end = '\r')
        q += 1
        out_string = ""
        out_string = str(i) + "\n"
        out_string += str(linkvectors.v1[i,0]) + "\n"
        out_string += str(linkvectors.v1[i,1]) + "\n"
        out_string += str(linkvectors.v1[i,2]) + "\n"
        out_string += str(linkvectors.v1[i,3]) + "\n"
        out_string += str(linkvectors.v1[i,4]) + "\n"
        out_string += str(linkvectors.v1[i,5]) + "\n"
        out_string += str(linkvectors.v1[i,6]) + "\n"
        out_string += str(linkvectors.v1[i,7]) + "\n"
        out_string += str(linkvectors.v1[i,8]) + "\n"
        out_string += "\n"
        out_file.write(out_string)

NameError: name 'linkvectors' is not defined